In [1]:
import importlib

import RktnChallenge.RktnModel
importlib.reload(RktnChallenge.RktnModel)
from RktnChallenge.RktnModel import ModelTrainer


from RktnChallenge.preprocessing.tokenizeString import tokenizeString
from RktnChallenge.preprocessing.filterStopWords import filterStopWords
from RktnChallenge.preprocessing.mergeFeatures import mergeFeatures
from RktnChallenge.preprocessing.mostOccur import mostOccur
from RktnChallenge.preprocessing.Dropper import Dropper
from RktnChallenge.preprocessing.TokenListToString import TokenListToString
from RktnChallenge.preprocessing.filterChar import filterChar

import RktnChallenge.preprocessing.lemmatize
importlib.reload(RktnChallenge.preprocessing.lemmatize)
from RktnChallenge.preprocessing.lemmatize import lemmatize

import RktnChallenge.preprocessing.regularExprSub
importlib.reload(RktnChallenge.preprocessing.regularExprSub)
from RktnChallenge.preprocessing.regularExprSub import regularExprSub

2023-09-07 14:09:29.322308: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-07 14:09:29.368122: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-07 14:09:30.040535: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-07 14:09:33.353792: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-07 1

In [2]:
RktnModel = ModelTrainer("data_tr_train.csv")
RktnModel.data=RktnModel.data[RktnModel.data.description.notna() ]

Removing StopWords

In [ ]:
On crop puis on merge

On merge et on crop ensuite

In [3]:
addendum = ['a','à','<p>','<b>','<div>','<em>','<br>','gt',
            'h','b','k','a','c','d','e','ex','Ndeg']

filtr = ['+','-','/','(',')',':',"'",'"',':','@''!','|','#','<','>','?',
         '1','2','3','4','5','6','7','8','9','0']

#on tokenise et retire les stopwords designation

desi_filter = filterChar("designation","designation", filtr)

desi_tokenize= tokenizeString("designation_tkn","designation")
desi_stopwords = filterStopWords("designation_tkn", "designation_tkn"
                                 ,lang=["english","french"],addendum=addendum)

desi_lemmatize = lemmatize("designation_tkn","designation_tkn")

#on tokenise et retire les stopwords description
desc_filter = filterChar("description","description", filtr)
desc_tokenize= tokenizeString("description_tkn","description")
desc_stopwords = filterStopWords("description_tkn", "description_tkn"
                                 ,lang=["english","french"],addendum=addendum)

desc_lemmatize = lemmatize("description_tkn","description_tkn")

merge_desi_desc = mergeFeatures("merged","designation_tkn","description_tkn")
desc_mostOccur= mostOccur("merged_desi_desc","merged",1500,1500)



toDrop = ["designation_tkn","description_tkn","merged"]
cleanDropper = Dropper(column_to_drop = toDrop)
toString = TokenListToString("merged_desi_desc","merged_desi_desc")

removeNumbers = regularExprSub("merged_desi_desc","merged_desi_desc",filtr='\d+')

RktnModel.initPreprocess()
RktnModel.addPreprocessStep("clean designation",desi_filter) # transformation en token
RktnModel.addPreprocessStep("tokenize designation",desi_tokenize) # transformation en token
RktnModel.addPreprocessStep("filter designation",desi_stopwords)  # filtrage des stopwords
RktnModel.addPreprocessStep("lemmitize designation",desi_lemmatize)  # filtrage des stopwords

RktnModel.addPreprocessStep("clean description",desc_filter) # transformation en token
RktnModel.addPreprocessStep("tokenize description",desc_tokenize) # transformation en token
RktnModel.addPreprocessStep("filter description",desc_stopwords)  # filtrage des stopwords
RktnModel.addPreprocessStep("lemmitize description",desc_lemmatize)  # filtrage des stopwords
RktnModel.addPreprocessStep("merge designation and desc",merge_desi_desc) # merge desi&desc
RktnModel.addPreprocessStep("desc reduce",desc_mostOccur)         # reduction taille desc
RktnModel.addPreprocessStep("tostring",toString) # merge desi&desc
RktnModel.addPreprocessStep("removeNumbers",removeNumbers) # merge desi&desc

#RktnModel.addPreprocessStep("cleaning",cleanDropper) # on clean les colonnes crées
RktnModel.preprocess()



[nltk_data] Downloading package stopwords to /home/pec/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/pec/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
RktnModel.saveCSV("data_train_desc_lemm.csv")

In [37]:
RktnModel = ModelTrainer("data_train_desc_lemm.csv")

In [38]:
removeNumbers = regularExprSub("merged_desi_desc","merged_desi_desc",filtr='\d+')

RktnModel.initPreprocess()
RktnModel.addPreprocessStep("removeNumbers",removeNumbers)
RktnModel.preprocess()

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,designation,description,productid,imageid,prdtypecode,imgname,desi_langue,tr_designation,desc_langue,tr_description,designation_tkn,description_tkn,merged,merged_desi_desc
0,0,36138,36138,36138,Nk Pro Racing - Jeu En Telechargement,A PROPOS DU JEU NKPro Racing dispose d'un mote...,1711734527,1100082580,2905,image_1100082580_product_1711734527.jpg,fr,Nk Pro Racing - Jeu En Telechargement,fr,A PROPOS DU JEU NKPro Racing dispose d'un mote...,"['nk', 'pro', 'racing', 'jeu', 'telechargemer']","['propos', 'jeu', 'nkpro', 'racing', 'disposer...","['nk', 'pro', 'racing', 'jeu', 'telechargemer'...",nk pro racing jeu telechargemer propos jeu nkp...
1,1,68630,68630,68630,Exit - Tombeau Du Pharaon,EXIT est un jeu qui reprend les sensations des...,2397542298,1139624392,1281,image_1139624392_product_2397542298.jpg,fr,Exit - Tombeau Du Pharaon,fr,EXIT est un jeu qui reprend les sensations des...,"['exit', 'tombeau', 'pharaon']","['exit', 'jeu', 'reprendre', 'sensation', 'esc...","['exit', 'tombeau', 'pharaon', 'exit', 'jeu', ...",exit tombeau pharaon exit jeu reprendre sensat...
2,2,36172,36172,36172,Bricolage Lettres En Mousse D'argent Alphabet ...,Bricolage lettres en mousse d'argent Alphabet ...,3979577549,1271697301,2060,image_1271697301_product_3979577549.jpg,fr,Bricolage Lettres En Mousse D'argent Alphabet ...,fr,Bricolage lettres en mousse d'argent Alphabet ...,"['bricolage', 'lettre', 'mousse', 'argent', 'a...","['bricolage', 'lettre', 'mousse', 'argent', 'a...","['bricolage', 'lettre', 'mousse', 'argent', 'a...",bricolage lettre mousse argent alphabet autoco...
3,3,9830,9830,9830,Nostalgique Fer Blanc Clockwork Chaine Jouet P...,Nostalgique fer blanc Clockwork chaine Jouet P...,4197657726,1313204688,1280,image_1313204688_product_4197657726.jpg,fr,Nostalgique Fer Blanc Clockwork Chaine Jouet P...,fr,Nostalgique fer blanc Clockwork chaine Jouet P...,"['nostalgique', 'fer', 'blanc', 'clockwork', '...","['nostalgique', 'fer', 'blanc', 'clockwork', '...","['nostalgique', 'fer', 'blanc', 'clockwork', '...",nostalgique fer blanc clockwork chaine jouet p...
4,5,25246,25246,25246,Maquette D'avion Multi-Rotor Dispositif De Lan...,Multi-rotorAerial Throwing Device Aerial Drone...,4172823765,1307772845,1300,image_1307772845_product_4172823765.jpg,fr,Maquette D'avion Multi-Rotor Dispositif De Lan...,fr,Multi-rotorAerial Throwing Device Aerial Drone...,"['maquette', 'avion', 'multi', 'rotor', 'dispo...","['multi', 'rotoraerial', 'throwing', 'device',...","['maquette', 'avion', 'multi', 'rotor', 'dispo...",maquette avion multi rotor dispositif lancemen...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44017,67925,80970,80970,80970,Voiture De Police Radio Commandee - Noir,Voiture de police radiocommandee police evolut...,3368768552,1209098122,1180,image_1209098122_product_3368768552.jpg,fr,Voiture De Police Radio Commandee - Noir,fr,Voiture de police radiocommandee police evolut...,"['voiture', 'police', 'radio', 'commandee', 'n...","['voiture', 'police', 'radiocommandee', 'polic...","['voiture', 'police', 'radio', 'commandee', 'n...",voiture police radio commandee noir voiture po...
44018,67928,44830,44830,44830,Kit d'entretien de piscine 9 pcs,Ce kit d'entretien de la piscine multifonction...,3929900621,1265216570,2583,image_1265216570_product_3929900621.jpg,fr,Kit d'entretien de piscine 9 pcs,fr,Ce kit d'entretien de la piscine multifonction...,"['kit', 'entretien', 'piscine', '9', 'pc']","['kit', 'entretien', 'piscine', 'multifonction...","['kit', 'entretien', 'piscine', '9', 'pc', 'ki...",kit entretien piscine pc kit entretien piscin...
44019,67929,19856,19856,19856,Professional Stainless Steel Strong Heavy Duty...,Professional Stainless Steel Strong Heavy Duty...,4061599082,1283698939,2582,image_1283698939_product_4061599082.jpg,fr,Professional Stainless Steel Strong Heavy Duty...,fr,Professional Stainless Steel Strong Heavy Duty...,"['professional', 'stainless', 

In [39]:
_X = RktnModel.data["merged_desi_desc"]

RktnModel.create_vectorizer(name="tokenizer", X = _X, ngram_range=(1,3),)

X_data = RktnModel.vectorizer_transform(_X)     


y = RktnModel.encodeLabel("prdtypecode")
label_size = RktnModel.getLabelSize()

import json
#on sauve le vectorizer
tokenizer_json = RktnModel.tokenizer.to_json()
with open('desc_tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

import joblib
joblib.dump(RktnModel.encoder, 'desc_encoder.pkl')

['desc_encoder.pkl']

In [40]:
from sklearn.model_selection import train_test_split

X_train = X_data
y_train = y
print(RktnModel.max_seq_length)

895


Definition d'un modele

In [47]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRUCell,RNN,GRU, Dense,Dropout,GlobalAveragePooling2D, LSTM, Flatten, Input
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.models import Model

_latent_dim = 600

vocab_size = RktnModel.vocab_size
print(vocab_size)
max_seq_length = RktnModel.max_seq_length
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=_latent_dim, input_length=max_seq_length, mask_zero=True),  
    Flatten(),   
    #GRU(units=64, dropout=0.2, recurrent_dropout=0.2),
    Dense(units=label_size, activation="softmax")
])


 
optimizer = AdamW(weight_decay=0.01, learning_rate=0.001) 
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy", metrics=['accuracy'])
model.summary()

104482
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 895, 600)          62689200  
                                                                 
 flatten_4 (Flatten)         (None, 537000)            0         
                                                                 


 dense_4 (Dense)             (None, 27)                14499027  
                                                                 
Total params: 77188227 (294.45 MB)
Trainable params: 77188227 (294.45 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [49]:
history = model.fit(X_train, y_train, epochs=6, batch_size=100, validation_split=0.2)

Epoch 1/6
353/353 [==============================] - 18s 52ms/step - loss: 0.0214 - accuracy: 0.9974 - val_loss: 0.4912 - val_accuracy: 0.8726
Epoch 2/6
353/353 [==============================] - 19s 53ms/step - loss: 0.0198 - accuracy: 0.9977 - val_loss: 0.5002 - val_accuracy: 0.8721
Epoch 3/6
353/353 [==============================] - 17s 47ms/step - loss: 0.0188 - accuracy: 0.9979 - val_loss: 0.4979 - val_accuracy: 0.8713
Epoch 4/6
353/353 [==============================] - 17s 48ms/step - loss: 0.0201 - accuracy: 0.9980 - val_loss: 0.4959 - val_accuracy: 0.8717
Epoch 5/6
353/353 [==============================] - 17s 49ms/step - loss: 0.0152 - accuracy: 0.9985 - val_loss: 0.5065 - val_accuracy: 0.8742
Epoch 6/6
353/353 [==============================] - 17s 48ms/step - loss: 0.0152 - accuracy: 0.9984 - val_loss: 0.5095 - val_accuracy: 0.8713


In [50]:


#y_pred_test = model.predict(X_test)
#y_pred_id = y_pred_test.argmax(axis=-1)

#RktnModel.evaluateTestResults(y_test,y_pred_id)


model.save("model_description.h5")

/home/pec/miniconda3/envs/tfGPU/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
